In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import csv
import pandas as pd
from datetime import datetime
import pytz 
from fake_useragent import UserAgent
import re
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def extract_tract_number(s):
    match = re.search(r'Census Tract (\d+\.*\d*)', s)
    return match.group(1) if match else None

In [3]:
census = pd.read_csv('/Users/ianduke/Desktop/MSDS_Fall_Module/Linear_Regression/Final_Project/Raw_Census_Data/2020_CA_Census_Info.csv', skiprows = 1, header = 0)

/var/folders/bp/vkhmx8nd2zxgz13drv5jw3fh0000gn/T/ipykernel_15514/4224179268.py:1: DtypeWarning: Columns (142,143,144,145,166,167,168,169,214,215,216,217,222,223,224,225,226,227,228,229,230,231,232,233,262,263,264,265,274,275,276,277,278,279,280,281,282,283,284,285,306,307,308,309,354,355,356,357,362,363,364,365,366,367,368,369,370,371,372,373,402,403,404,405,414,415,416,417,418,419,420,421,422,423,424,425,446,447,448,449,502,503,504,505,506,507,508,509,510,511,512,513,542,543,544,545,558,559,560,561) have mixed types. Specify dtype option on import or set low_memory=False.
  census = pd.read_csv('/Users/ianduke/Desktop/MSDS_Fall_Module/Linear_Regression/Final_Project/Raw_Census_Data/2020_CA_Census_Info.csv', skiprows = 1, header = 0)


In [4]:
columns_of_interest = ['Geographic Area Name', 'Estimate!!Total!!Population 16 years and over', 'Estimate!!Unemployment rate!!Population 16 years and over','Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years', 'Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years', 'Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate', 'Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency)', "Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree", "Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher"]      

In [5]:
filtered_census = census[columns_of_interest]

In [3]:
filtered_census['Geographic Area Name'] = filtered_census['Geographic Area Name'].apply(extract_tract_number)

NameError: name 'filtered_census' is not defined

In [7]:
filtered_census = filtered_census[filtered_census['Geographic Area Name'].notnull()]

In [8]:
filtered_census['Geographic Area Name'] = filtered_census['Geographic Area Name'].astype(float)

In [9]:
filtered_census = filtered_census.drop_duplicates(subset='Geographic Area Name', keep='first')

## Now that we've garnered our first set of potential predictors, let's join with our incarceration data!

In [4]:
tract_incarceration = pd.read_csv("/Users/ianduke/Desktop/MSDS_Fall_Module/Linear_Regression/Final_Project/tract_level_incarceration_data.csv")

In [11]:
tract_incarceration = tract_incarceration.drop_duplicates(subset = "California Census tracts", keep = "first")


In [12]:
merged_data = pd.merge(tract_incarceration, filtered_census,
                      left_on = "California Census tracts",
                      right_on = "Geographic Area Name",
                      how = "inner")

In [13]:
merged_data

,FIPS Code 2020,California Census tracts,Number of people in state prison from each Census tract 2020,"Census population, 2020","Total population, 2020","Imprisonment rate per 100,000",Geographic Area Name,Estimate!!Total!!Population 16 years and over,Estimate!!Unemployment rate!!Population 16 years and over,Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher
0,6001400100,4001.00,3,"3,038","3,041",99,4001.00,2511,1.0,65,1644,24,18,141,1461
1,6001400200,4002.00,1,"2,001","2,002",50,4002.00,1669,7.9,22,1174,18,46,95,1015
2,6001400300,4003.00,1,"5,504","5,505",18,4003.00,4317,3.4,202,3193,82,87,463,2561
3,6001400400,4004.00,6,"4,112","4,118",146,4004.00,3361,2.5,99,2705,78,165,362,2100
4,6001400500,4005.00,15,"3,644","3,659",410,4005.00,3588,6.4,111,2409,30,293,294,1792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7364,6113011210,112.10,6,"2,738","2,744",219,112.10,2287,1.6,228,1370,164,247,616,343
7365,6113011211,112.11,4,"1,653","1,657",241,112.11,1253,0.0,35,837,10,198,195,434
7366,6113011304,113.04,4,"1,361","1,365",293,113.04,1156,5.0,102,877,152,177,438,110
7367,6115040301,403.01,27,"3,600","3,627",744,403.01,3040,9.2,162,2398,382,690,892,434


# Hooray! Now let's add racial pluralities

In [14]:
race = pd.read_csv('/Users/ianduke/Desktop/MSDS_Fall_Module/Linear_Regression/Final_Project/Raw_Census_Data/DECENNIALPL2020.P1-Data.csv', skiprows = 1, header = 0)

In [15]:
filtered_race = race[[col for col in race.columns if col.startswith(" !!Total") or col.startswith('Geographic')]]

In [18]:
#calculate the proportions for each racial category
race_dataframe = pd.DataFrame()
race_dataframe['Tract'] = filtered_race['Geographic Area Name']

columns_to_proportionalize = [' !!Total:!!Population of one race:!!White alone', ' !!Total:!!Population of one race:!!Black or African American alone',' !!Total:!!Population of one race:!!American Indian and Alaska Native alone', ' !!Total:!!Population of one race:!!Asian alone', ' !!Total:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone', ' !!Total:!!Population of one race:!!Some Other Race alone']

#Remove any rows with populations less than 1
filtered_race = filtered_race[filtered_race[' !!Total:'] >= 1].reset_index(drop=True)


def proportion(column):
    col_name = column+"_PROPORTION"
    race_dataframe[col_name] = ""
    for i in range(len(filtered_race)):
        race_dataframe[col_name][i] = filtered_race[column][i]/filtered_race[' !!Total:'][i]

for column in columns_to_proportionalize:
    proportion(column)

In [19]:
filtered_race

,Geographic Area Name,!!Total:,!!Total:!!Population of one race:,!!Total:!!Population of one race:!!White alone,!!Total:!!Population of one race:!!Black or African American alone,!!Total:!!Population of one race:!!American Indian and Alaska Native alone,!!Total:!!Population of one race:!!Asian alone,!!Total:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone,!!Total:!!Population of one race:!!Some Other Race alone,!!Total:!!Population of two or more races:,...,!!Total:!!Population of two or more races:!!Population of four races:!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of six races:,!!Total:!!Population of two or more races:!!Population of six races:!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race
0,"Census Tract 4001, Alameda County, California",3038,2645,1870,144,2,538,11,80,393,...,0,0,0,0,0,0,0,0,0,0
1,"Census Tract 4002, Alameda County, California",2001,1716,1421,40,3,207,4,41,285,...,0,2,0,2,0,0,0,0,0,0
2,"Census Tract 4003, Alameda County, California",5504,4772,3413,561,26,554,27,191,732,...,0,2,1,0,0,0,1,0,0,0
3,"Census Tract 4004, Alameda County, California",4112,3595,2765,289,12,399,8,122,517,...,0,0,0,0,0,0,0,0,0,0
4,"Census Tract 4005, Alameda County, California",3644,3086,1957,625,24,307,5,168,558,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9092,"Census Tract 409.02, Yuba County, California",2412,2003,1572,214,12,108,25,72,409,...,0,4,2,2,0,0,0,0,0,0
9093,"Census Tract 410.01, Yuba County, California",3594,3205,2623,26,62,124,12,358,389,...,0,2,0,2,0,0,0,0,0,0
9094,"Census Tract 410.02, Yuba County, California",3935,3562,3225,22,72,90,6,147,373,...,0,2,0,2,0,0,0,0,0,0
9095,"Census Tract 411.01, Yuba County, California",3187,2817,2552,33,80,51,15,86,370,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#Clean up the tract column
race_dataframe['Tract'] = race_dataframe['Tract'].apply(extract_tract_number)


In [21]:
#Remove duplicates
race_dataframe = race_dataframe.drop_duplicates(subset='Tract', keep='first')

In [22]:
race_dataframe['Tract'] = race_dataframe['Tract'].astype('float')

In [23]:
# Set 'Tract' column as the index
race_df = race_dataframe.set_index('Tract')

race_df = race_df.astype(float, errors='ignore')



# Get the race column that has the max value for each row
plurality_race = race_df.idxmax(axis=1)

# Create a new DataFrame with the results
plurality_df = pd.DataFrame({
    'Tract': plurality_race.index,
    'Plurality Race': plurality_race.values
}).reset_index(drop=True)

In [24]:
plurality_dataframe = pd.DataFrame({'Tract': plurality_race.index, 'Plurality Race': plurality_race.values})

In [25]:
def extract_race(entry):
    parts = entry.split("!!")
    if "alone" in parts[-1]:  # To make sure we are in the right section of the string.
        return parts[-1].split(" ")[0]  # Taking the first word of the race description
    return entry

In [26]:
# Apply the function to the 'Plurality Race' column
plurality_dataframe['Plurality Race'] = plurality_dataframe['Plurality Race'].apply(extract_race)

In [27]:
for i in range(len(plurality_dataframe)):
    if plurality_dataframe['Plurality Race'][i]=='Some':
        plurality_dataframe['Plurality Race'][i] = 'Some Other Race'
    if plurality_dataframe['Plurality Race'][i]=='Black':
        plurality_dataframe['Plurality Race'][i] = 'Black or African American'
    if plurality_dataframe['Plurality Race'][i]=='American Indian and Alaska Native':
        plurality_dataframe['Plurality Race'][i] = 'Some Other Race'

/var/folders/bp/vkhmx8nd2zxgz13drv5jw3fh0000gn/T/ipykernel_15514/1567484516.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plurality_dataframe['Plurality Race'][i] = 'Black or African American'
/var/folders/bp/vkhmx8nd2zxgz13drv5jw3fh0000gn/T/ipykernel_15514/1567484516.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plurality_dataframe['Plurality Race'][i] = 'Some Other Race'


In [28]:
# Merge the two DataFrames based on the specified columns
merged_data = pd.merge(race_dataframe, merged_data, 
                       left_on="Tract", 
                       right_on="California Census tracts", 
                       how="inner")

In [29]:
#Merge the two DataFrames based on the specified columns
merged_data = pd.merge(plurality_dataframe, merged_data, 
                       left_on="Tract", 
                       right_on="California Census tracts", 
                       how="inner")

In [31]:
merged_data

,Tract_x,Plurality Race,Tract_y,!!Total:!!Population of one race:!!White alone_PROPORTION,!!Total:!!Population of one race:!!Black or African American alone_PROPORTION,!!Total:!!Population of one race:!!American Indian and Alaska Native alone_PROPORTION,!!Total:!!Population of one race:!!Asian alone_PROPORTION,!!Total:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone_PROPORTION,!!Total:!!Population of one race:!!Some Other Race alone_PROPORTION,FIPS Code 2020,...,"Imprisonment rate per 100,000",Geographic Area Name,Estimate!!Total!!Population 16 years and over,Estimate!!Unemployment rate!!Population 16 years and over,Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher
0,4001.00,White,4001.00,0.615537,0.0474,0.000658,0.17709,0.003621,0.026333,6001400100,...,99,4001.00,2511,1.0,65,1644,24,18,141,1461
1,4002.00,White,4002.00,0.710145,0.01999,0.001499,0.103448,0.001999,0.02049,6001400200,...,50,4002.00,1669,7.9,22,1174,18,46,95,1015
2,4003.00,White,4003.00,0.620094,0.101926,0.004724,0.100654,0.004906,0.034702,6001400300,...,18,4003.00,4317,3.4,202,3193,82,87,463,2561
3,4004.00,White,4004.00,0.672422,0.070282,0.002918,0.097033,0.001946,0.029669,6001400400,...,146,4004.00,3361,2.5,99,2705,78,165,362,2100
4,4005.00,White,4005.00,0.537047,0.171515,0.006586,0.084248,0.001372,0.046103,6001400500,...,410,4005.00,3588,6.4,111,2409,30,293,294,1792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7352,112.10,White,112.10,0.339664,0.011322,0.016801,0.167275,0.00767,0.254565,6113011210,...,219,112.10,2287,1.6,228,1370,164,247,616,343
7353,112.11,White,112.11,0.410163,0.017544,0.008469,0.128252,0.00605,0.196007,6113011211,...,241,112.11,1253,0.0,35,837,10,198,195,434
7354,113.04,White,113.04,0.396032,0.005878,0.047024,0.017634,0.00147,0.351212,6113011304,...,293,113.04,1156,5.0,102,877,152,177,438,110
7355,403.01,White,403.01,0.465,0.033333,0.021944,0.134722,0.004444,0.175833,6115040301,...,744,403.01,3040,9.2,162,2398,382,690,892,434


## Great! Now let's add one more categorical predictor: urban vs rural tracts

In [32]:
#Now lets add our rural vs urban data
urban_rural = pd.read_csv('/Users/ianduke/Desktop/MSDS_Fall_Module/Linear_Regression/Final_Project/Raw_Census_Data/DECENNIALDHC2020.H2-Data.csv', skiprows = 1, header = 0 )

In [33]:
#Clean up the tract column
urban_rural['Geographic Area Name'] = urban_rural['Geographic Area Name'].apply(extract_tract_number)

In [34]:
filtered_urban = urban_rural[[col for col in urban_rural.columns if col.startswith(" !!Total") or col.startswith('Geographic')]]

In [35]:
filtered_urban = filtered_urban[["Geographic Area Name", ' !!Total:!!Urban', ' !!Total:!!Rural']]

In [36]:
# Create a new column 'Area_Type' which will have either 'Urban' or 'Rural' based on the comparison
filtered_urban['Area_Type'] = np.where(filtered_urban[' !!Total:!!Urban'] > filtered_urban[' !!Total:!!Rural'], 'Urban', 'Rural')

In [37]:
filtered_urban = filtered_urban[filtered_urban['Geographic Area Name'].notnull()]

In [38]:
filtered_urban = filtered_urban[['Geographic Area Name', 'Area_Type']]

In [39]:
filtered_urban['Geographic Area Name'] = filtered_urban['Geographic Area Name'].astype(float)

In [40]:
filtered_urban = filtered_urban.drop_duplicates(subset='Geographic Area Name', keep='first')

In [43]:
# Merge the two DataFrames based on the specified columns
merged_data = pd.merge(filtered_urban, merged_data, 
                       left_on="Geographic Area Name", 
                       right_on="Tract_x", 
                       how="inner")

## Finally, let's add some information about the total number of people living in poverty in each census tract

In [45]:
poverty = pd.read_csv('/Users/ianduke/Desktop/MSDS_Fall_Module/Linear_Regression/Final_Project/Raw_Census_Data/ACSST5Y2020.S1701-Data.csv',skiprows = 1, header = 0)

In [46]:
poverty_filtered = poverty[['Geographic Area Name', 'Estimate!!Percent below poverty level!!Population for whom poverty status is determined']]

In [47]:
poverty_filtered['Geographic Area Name'] = poverty_filtered['Geographic Area Name'].apply(extract_tract_number)

/var/folders/bp/vkhmx8nd2zxgz13drv5jw3fh0000gn/T/ipykernel_15514/3243305639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poverty_filtered['Geographic Area Name'] = poverty_filtered['Geographic Area Name'].apply(extract_tract_number)


In [48]:
poverty_filtered = poverty_filtered.drop_duplicates(subset='Geographic Area Name', keep='first')

In [49]:
poverty_filtered['Geographic Area Name'] = poverty_filtered['Geographic Area Name'].astype(float)

In [50]:
poverty_filtered = poverty_filtered[poverty_filtered['Geographic Area Name'].notnull()]

In [51]:
poverty_filtered

,Geographic Area Name,Estimate!!Percent below poverty level!!Population for whom poverty status is determined
1,4001.00,4.4
2,4002.00,5.8
3,4003.00,5.1
4,4004.00,9.4
5,4005.00,9.8
...,...,...
9103,112.10,13.0
9104,112.11,3.1
9108,113.04,14.9
9114,403.01,39.4


In [52]:
# Merge the two DataFrames based on the specified columns
merged_data = pd.merge(poverty_filtered, merged_data, 
                       left_on="Geographic Area Name", 
                       right_on="Geographic Area Name_x", 
                       how="inner")

In [53]:
merged_data

,Geographic Area Name,Estimate!!Percent below poverty level!!Population for whom poverty status is determined,Geographic Area Name_x,Area_Type,Tract_x,Plurality Race,Tract_y,!!Total:!!Population of one race:!!White alone_PROPORTION,!!Total:!!Population of one race:!!Black or African American alone_PROPORTION,!!Total:!!Population of one race:!!American Indian and Alaska Native alone_PROPORTION,...,"Imprisonment rate per 100,000",Geographic Area Name_y,Estimate!!Total!!Population 16 years and over,Estimate!!Unemployment rate!!Population 16 years and over,Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Estimate!!Total!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher
0,4001.00,4.4,4001.00,Urban,4001.00,White,4001.00,0.615537,0.0474,0.000658,...,99,4001.00,2511,1.0,65,1644,24,18,141,1461
1,4002.00,5.8,4002.00,Urban,4002.00,White,4002.00,0.710145,0.01999,0.001499,...,50,4002.00,1669,7.9,22,1174,18,46,95,1015
2,4003.00,5.1,4003.00,Urban,4003.00,White,4003.00,0.620094,0.101926,0.004724,...,18,4003.00,4317,3.4,202,3193,82,87,463,2561
3,4004.00,9.4,4004.00,Urban,4004.00,White,4004.00,0.672422,0.070282,0.002918,...,146,4004.00,3361,2.5,99,2705,78,165,362,2100
4,4005.00,9.8,4005.00,Urban,4005.00,White,4005.00,0.537047,0.171515,0.006586,...,410,4005.00,3588,6.4,111,2409,30,293,294,1792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7352,112.10,13.0,112.10,Urban,112.10,White,112.10,0.339664,0.011322,0.016801,...,219,112.10,2287,1.6,228,1370,164,247,616,343
7353,112.11,3.1,112.11,Urban,112.11,White,112.11,0.410163,0.017544,0.008469,...,241,112.11,1253,0.0,35,837,10,198,195,434
7354,113.04,14.9,113.04,Urban,113.04,White,113.04,0.396032,0.005878,0.047024,...,293,113.04,1156,5.0,102,877,152,177,438,110
7355,403.01,39.4,403.01,Urban,403.01,White,403.01,0.465,0.033333,0.021944,...,744,403.01,3040,9.2,162,2398,382,690,892,434


## We've collected all information for our possible predictors! Let's save this to a dataframe for later model selection

In [54]:
merged_data.to_csv("final_merged_data.csv", index=False)